In [ ]:
import torch
import torchvision
from torchsummary import summary
import warnings
# Suppress FutureWarning specifically from torch.load
warnings.filterwarnings("ignore", category=FutureWarning)
import numpy as np
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

################### GPU ##########################

device = ("cuda" if torch.cuda.is_available() else "cpu")

if torch.backends.cudnn.is_available()==True:
    print('CUDNN is available! ')
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True

try:
    import torch_directml
    device = torch_directml.device()
    print("DirectML is available, using DirectML")
except:
    print("DirectML is not available, using CPU/GPU")

##################################################

import windturbine_final as wt

CUDNN is available! 
DirectML is not available, using CPU/GPU


c:\Users\bjark\OneDrive - Aarhus universitet\InTooDeep_project\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
ResNet34 = torchvision.models.resnet34(weights=False)
ResNet34.fc = torch.nn.Linear(in_features=512,out_features=2,bias=True)
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet34.to(device)
# Print summary for a (3, 300, 300) input
summary(model, input_size=(3, 300, 300), device=device.type)

In [8]:
def transform(image):
    image = np.array(image)
    image = image[225:525,490:790]
    
    # Convert back to PIL Image for further transforms
    image = Image.fromarray(image)
    
    # Compose transformations
    transform = transforms.Compose([
        transforms.ToTensor(),  # Convert to tensor
    ])
    return transform(image)

def transform_noise(image):
    # Get either 0 or 1
    noise = np.random.randint(0, 2)
    if noise == 0:
        # Get 2 random numbers between -50 and 50
        x,y = np.random.randint(-50, 50, 2)

        # Convert image to array
        image = np.array(image)
        image = image[225+x:525+x,490+y:790+y]

        # Convert back to PIL Image for further transforms
        image = Image.fromarray(image)

        # Compose transformations
        transform = transforms.Compose([
            transforms.ToTensor(),  # Convert to tensor
        ])
    else:
        # Get a random number between 400 and 720
        x = np.random.randint(400, 720)
        transform = transforms.Compose([
            transforms.CenterCrop(x),
            transforms.Resize(300),
            transforms.ToTensor(),
        ])
    return transform(image)


### Parameters ResNet34

In [ ]:
angle_type = "both"
batch_size = 16
images_num = 1
base_angle_range = [0,360]
model = ResNet34
lr = 1e-2
epochs = 40
accu_th = [20,10,5]
model_name = "final_models/ResNet34_final_noise.pth"

wind_dataset = wt.WindTurbineDataset(csv_file='rotations_w_images_long.csv', root_dir='data/', 
                                     images_num=images_num, transform=transform_noise, angle_type=angle_type, base_angle_range=base_angle_range)
print(f"Dataset size: {len(wind_dataset)}")

train_dataset, test_dataset = wt.WindTurbineDataloader.train_test_split(wind_dataset, test_size=0.2)
trainloader = wt.WindTurbineDataloader.dataloader(train_dataset, batch_size=batch_size, shuffle=True)
testloader = wt.WindTurbineDataloader.dataloader(test_dataset, batch_size=batch_size, shuffle=True)

try:
    model.load_state_dict(torch.load(model_name, map_location=torch.device("cpu")))
    print("Model loaded successfully")
except:
    print("Model not found, training from scratch")
model = model.to(device)

criterion = wt.AngularVectorLoss()

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#Scheduler
schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=np.sqrt(0.1), patience=2, threshold=0.0001)

#Trainer
trainer = wt.Trainer(model, trainloader, testloader, criterion, optimizer,device, 
                     epochs=epochs, accu_th=accu_th, angle_type=angle_type, 
                     schedular=schedular, minimal=False)


Dataset size: 18000
Model not found, training from scratch


### ResNet34 Training

In [ ]:
model = trainer.train_model()

# Plot the training and testing loss
plt.figure(figsize=(10, 5))
# Make subplot with loss and accuracy
plt.subplot(1, 2, 1)
plt.plot(trainer.train_loss, label="Train Loss")
plt.plot(trainer.test_loss, label="Test Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
if not trainer.minimal:
    # Make subplot with loss and accuracy
    plt.subplot(1, 2, 2)
    plt.plot(trainer.train_accuracy, label="Train Accuracy")
    plt.plot(trainer.test_accuracy, label="Test Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid()
plt.show()

### Save Model

In [ ]:
torch.save(model.to("cpu").state_dict(), model_name)
print("Model saved successfully")

### Test Model

In [ ]:
# Test the model
results = trainer.test_model(model.to(device), wind_dataset, angle_type=angle_type)
# Sort the results by base angle
results_sorted = results.sort_values(by="Base_Angle")

# Plot the results
if angle_type == "both" or angle_type == "base_angle":
    plt.figure(figsize=(10, 3))
    plt.stem(results_sorted["Base_Angle"], results_sorted["Base_Angle_Error"], label="Base Angle Error")
    plt.xlabel("Base Angle")
    plt.ylabel("Error")
    plt.legend()
    plt.grid()
    plt.show()
if angle_type == "both" or angle_type == "blade_angle":
    plt.figure(figsize=(10, 3))
    plt.stem(results_sorted["Base_Angle"], results_sorted["Blade_Angle_Error"], label="Blade Angle Error")
    plt.xlabel("Blade Angle")
    plt.ylabel("Error")
    plt.legend()
    plt.grid()
    plt.show()